# 심층 신경망

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/7-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 2개의 층

In [1]:
####################################################################################################
# 패션 MNIST 데이터셋을 읽어 훈련셋과 테스트셋에 저장합니다.
# --------------------------------------------------------------------------------------------------
# load_data() 메서드는 두 종류의 데이터셋을 한번에 반환하므로 분할 절차를 생략할 수 있어 편리합니다.
####################################################################################################
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
####################################################################################################
# 데이터의 스케일을 조정하고 1차원 배열로 변환합니다.
# --------------------------------------------------------------------------------------------------
# 0~255의 값을 0~1로 조정합니다.
# 이미지의 경우 255로 나눠주기만 하면 됩니다. MinMaxScaler()를 사용한 것과 동일한 결과입니다.
# 2차원 배열의 형태를 가진 하나의 샘플을 1차원 배열로 변환해주면, 784개(28*28) 값이 1개 행으로 변환됩니다.
####################################################################################################
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [3]:
####################################################################################################
# 신경망에 포함될 밀집층을 생성합니다.
# --------------------------------------------------------------------------------------------------
# 여기서는 은닉층, 출력층 각각 1개에 해당하는 Dense Layer 객체를 생성해야 합니다.
# 입력층 다음의 최초 은닉층에 대해 input_shape을 설정해주면 됩니다.
# 은닉층이 없는 경우 출력층에 설정하면 됩니다.
####################################################################################################
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

## 심층 신경망 만들기

In [4]:
####################################################################################################
# 미리 생성한 밀집층 객체를 리스트에 포함시켜 모델을 생성합니다.
# --------------------------------------------------------------------------------------------------
# 밀집층의 순서는 입력층에서 가까운 순서로 은닉층, 그리고 마지막에 출력층을 배치합니다.
####################################################################################################
model = keras.Sequential([dense1, dense2])

In [5]:
####################################################################################################
# 모델에 대한 정보를 확인합니다.
# --------------------------------------------------------------------------------------------------
# 파라미터는 훈련에 의해 학습되는 값으로서, 가중치와 절편이 포함됩니다.
####################################################################################################
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


## 층을 추가하는 다른 방법

In [6]:
####################################################################################################
# 밀집층 객체를 생성하는 동시에 리스트에 포함시켜 모델을 생성합니다.
####################################################################################################
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [7]:
####################################################################################################
# 모델에 대한 정보를 확인합니다.
# --------------------------------------------------------------------------------------------------
# 파라미터는 훈련에 의해 학습되는 값으로서, 가중치와 절편이 포함됩니다.
####################################################################################################
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [8]:
####################################################################################################
# 모델 객체를 만들어 놓고, 층을 새로 생성하면서 모델에 추가시키는 방법으로 모델을 생성합니다.
####################################################################################################
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [9]:
####################################################################################################
# 모델에 대한 정보를 확인합니다.
# --------------------------------------------------------------------------------------------------
# 파라미터는 훈련에 의해 학습되는 값으로서, 가중치와 절편이 포함됩니다.
# 3가지 모델 생성 방법이 모두 동일한 결과를 보임을 확인할 수 있습니다.
####################################################################################################
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
####################################################################################################
# 인공신경망 훈련을 위한 설정값(손실함수와 성능지표)을 입력합니다.
# --------------------------------------------------------------------------------------------------
# 이진 분류의 경우 손실함수를 'binary_crossentropy'로, 다중 분류의 경우 'categorical_crossentropy'로
# 설정합니다.
# 타깃 데이터의 레이블이 원-핫 인코딩된 경우 'sparse_'를 손실함수명 앞에 추가하면 됩니다.
# 성능지표에 'accuracy'를 사용하면 정확도를 사용합니다.
####################################################################################################
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 1s 931us/step - loss: 0.5657 - accuracy: 0.8068
Epoch 2/5
1500/1500 [==============================] - 1s 927us/step - loss: 0.4071 - accuracy: 0.8533
Epoch 3/5
1500/1500 [==============================] - 1s 943us/step - loss: 0.3723 - accuracy: 0.86540s
Epoch 4/5
1500/1500 [==============================] - 1s 891us/step - loss: 0.3503 - accuracy: 0.8737
Epoch 5/5
1500/1500 [==============================] - 1s 829us/step - loss: 0.3335 - accuracy: 0.8780


## 렐루 활성화 함수

In [11]:
####################################################################################################
# 은닉층에 렐루(ReLU) 활성화 함수를 사용합니다.
# --------------------------------------------------------------------------------------------------
# 이미지 데이터에는 렐루 함수가 좀 더 높은 성능을 내는 것으로 알려져 있습니다.
# Flatten 층을 추가하게 되면, 입력 데이터의 1차원 변환을 수행해줍니다.
####################################################################################################
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [12]:
####################################################################################################
# 모델에 대한 정보를 확인합니다.
# --------------------------------------------------------------------------------------------------
# 파라미터는 훈련에 의해 학습되는 값으로서, 가중치와 절편이 포함됩니다.
# 3가지 모델 생성 방법이 모두 동일한 결과를 보임을 확인할 수 있습니다.
####################################################################################################
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [13]:
####################################################################################################
# 데이터의 스케일을 조정하고 1차원 배열로 변환합니다.
# --------------------------------------------------------------------------------------------------
# 0~255의 값을 0~1로 조정합니다.
# 이미지의 경우 255로 나눠주기만 하면 됩니다. MinMaxScaler()를 사용한 것과 동일한 결과입니다.
# 2차원 배열의 형태를 가진 하나의 샘플을 1차원 배열로 변환해주면, 784개(28*28) 값이 1개 행으로 변환됩니다.
####################################################################################################
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [14]:
####################################################################################################
# 인공신경망 훈련을 위한 설정값(손실함수와 성능지표)을 입력하고, 훈련을 실시합니다.
# --------------------------------------------------------------------------------------------------
# 이진 분류의 경우 손실함수를 'binary_crossentropy'로, 다중 분류의 경우 'categorical_crossentropy'로
# 설정합니다.
# 타깃 데이터의 레이블이 원-핫 인코딩된 경우 'sparse_'를 손실함수명 앞에 추가하면 됩니다.
# 성능지표에 'accuracy'를 사용하면 정확도를 사용합니다.
####################################################################################################
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5353 - accuracy: 0.8132
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3950 - accuracy: 0.8605
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3598 - accuracy: 0.8718
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3343 - accuracy: 0.8804
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3209 - accuracy: 0.8865: 0s - loss: 0.3248 - 


In [15]:
####################################################################################################
# 모델의 성능을 검증합니다.
# --------------------------------------------------------------------------------------------------
# 모델이 과적합되었는지를 확인해야 하므로 검증 데이터를 활용합니다.
####################################################################################################
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 608us/step - loss: 0.3700 - accuracy: 0.8808


[0.3699990212917328, 0.8808333277702332]

## 옵티마이저

In [16]:
####################################################################################################
# 케라스는 모델의 파라미터가 최적값에 잘 수렴하도록 다양한 옵티마이저(경사하강법 알고리즘)를 제공합니다.
# --------------------------------------------------------------------------------------------------
# 각각을 설정하는 방법을 배워보도록 합니다.
# 옵티마이저도 하이퍼파라미터입니다. 
# 주어진 문제에 따라 최적의 옵티마이저는 달라질 수 있고, 이는 다양한 값을 넣고 훈련하면서 발견해야 합니다.
####################################################################################################
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 1s 714us/step - loss: 0.2669 - accuracy: 0.9043
Epoch 2/5
1500/1500 [==============================] - 1s 663us/step - loss: 0.2576 - accuracy: 0.9074
Epoch 3/5
1500/1500 [==============================] - 1s 690us/step - loss: 0.2537 - accuracy: 0.9090
Epoch 4/5
1500/1500 [==============================] - 1s 763us/step - loss: 0.2509 - accuracy: 0.9099
Epoch 5/5
375/375 [==============================] - 0s 661us/step - loss: 0.3189 - accuracy: 0.8914


[0.31891411542892456, 0.8914166688919067]

In [17]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2462 - accuracy: 0.9114
Epoch 2/5
1500/1500 [==============================] - 1s 845us/step - loss: 0.2441 - accuracy: 0.9110
Epoch 3/5
1500/1500 [==============================] - 1s 821us/step - loss: 0.2425 - accuracy: 0.9132
Epoch 4/5
1500/1500 [==============================] - 1s 881us/step - loss: 0.2408 - accuracy: 0.9128
Epoch 5/5
375/375 [==============================] - 0s 632us/step - loss: 0.3136 - accuracy: 0.8925


[0.31362584233283997, 0.8924999833106995]

In [18]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 1s 791us/step - loss: 0.2375 - accuracy: 0.9135
Epoch 2/5
1500/1500 [==============================] - 1s 889us/step - loss: 0.2357 - accuracy: 0.9143
Epoch 3/5
1500/1500 [==============================] - 1s 997us/step - loss: 0.2347 - accuracy: 0.91510s - loss: 0.2345 - accuracy
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2330 - accuracy: 0.9147
Epoch 5/5
375/375 [==============================] - 0s 701us/step - loss: 0.3118 - accuracy: 0.8923


[0.311763197183609, 0.8922500014305115]

In [19]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2306 - accuracy: 0.9171
Epoch 2/5
1500/1500 [==============================] - 1s 881us/step - loss: 0.2289 - accuracy: 0.9168
Epoch 3/5
1500/1500 [==============================] - 1s 871us/step - loss: 0.2277 - accuracy: 0.9171
Epoch 4/5
1500/1500 [==============================] - 1s 821us/step - loss: 0.2265 - accuracy: 0.9180
Epoch 5/5
375/375 [==============================] - 0s 611us/step - loss: 0.3103 - accuracy: 0.8942


[0.3102615475654602, 0.8941666483879089]

In [20]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 1s 845us/step - loss: 0.2208 - accuracy: 0.9203
Epoch 2/5
1500/1500 [==============================] - 1s 920us/step - loss: 0.2199 - accuracy: 0.9202
Epoch 3/5
1500/1500 [==============================] - 1s 883us/step - loss: 0.2195 - accuracy: 0.9205
Epoch 4/5
1500/1500 [==============================] - 1s 919us/step - loss: 0.2192 - accuracy: 0.9206
Epoch 5/5
375/375 [==============================] - 0s 587us/step - loss: 0.3076 - accuracy: 0.8949


[0.30761417746543884, 0.8949166536331177]

In [21]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2904 - accuracy: 0.8954
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2825 - accuracy: 0.8992
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2764 - accuracy: 0.9009
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2718 - accuracy: 0.9044
Epoch 5/5
375/375 [==============================] - 0s 584us/step - loss: 0.3942 - accuracy: 0.8795


[0.39421117305755615, 0.8794999718666077]

In [22]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [23]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5301 - accuracy: 0.8140
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3977 - accuracy: 0.8576: 0s -
Epoch 3/5
1500/1500 [==============================] - 1s 955us/step - loss: 0.3578 - accuracy: 0.8699
Epoch 4/5
1500/1500 [==============================] - 1s 809us/step - loss: 0.3308 - accuracy: 0.8786
Epoch 5/5
375/375 [==============================] - 0s 621us/step - loss: 0.3363 - accuracy: 0.8785


[0.3363282382488251, 0.8784999847412109]